In [7]:
import requests
import re
from bs4 import BeautifulSoup as bs
import time
import pickle

In [3]:
def salary_parser(salary):
    salary = salary.replace('\xa0', '')
    salary_dict = {
        'min':None,
        'max':None,
        'currency':None,
        'comment':None
    }
    
    if 'от ' in salary.lower():
        salary_dict['comment'] = 'От'
        salary = salary.replace(' ', '')
        salary_dict['min'] = re.search('[0-9]+', salary)[0]
    elif 'до ' in salary.lower():
        salary_dict['comment'] = 'До'
        salary = salary.replace(' ', '')
        salary_dict['max'] = re.search('[0-9]+', salary)[0]
    elif '—' in salary:
        salary_dict['currency'] = salary.split()[-1]
        salary = salary.replace(' ','')
        search = re.findall('[0-9]+', salary)
        salary_dict['min'], salary_dict['max'] = search[0], search[1]
    else:
        
        if re.findall('[0-9]+', salary):
            salary = salary.replace(' ','')
            search = re.findall('[0-9]+', salary)
            salary_dict['min'], salary_dict['max'] = search[0], search[0]
        else:
            salary_dict['comment'] = salary
    
    return salary_dict

In [4]:
def job_searcher(query, max_pages=20):
    """
    Функция по поиску работы на работа.ру
       : param query : Желаемая должность
       : param max_pages : Максимальное количество страниц, которое запросит алгоритм
    """
    
    
    headers = {
        'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36'
    }
    url = 'https://nn.rabota.ru/vacancy'
    page = 1
    results = []

    while True:
        
        if page > max_pages:
            break
        params = {
            'query' : query,
            'sort' : 'relevance',
            'page' : page
        }
        response = requests.get(url, params, headers=headers)
        
        print(f'Выполняется запрос, страница: {page}')
        
        if not response.ok:
            break

        soup = bs(response.text, 'html.parser')

        vacancies = soup.find_all('h3', attrs={'class':'vacancy-preview-card__title'})
        if not vacancies:
            break

        platform = url.split(sep='/')[2]

        for vacancy in vacancies:
            title = vacancy.find('a').text.strip()
            salary = vacancy.parent.find('div', {'class':'vacancy-preview-card__salary'}).find('a').text.strip()
            ref = url + vacancy.find('a')['href']
            results.append({
                'ref':ref,
                'salary_raw':salary,
                'salary':salary_parser(salary),
                'title':title,
                'platform':platform
            })
        page += 1
        time.sleep(3)
    print('Выполнение завершено.')    
    return results

In [5]:
vacancies = job_searcher('бухгалтер', 4)

Выполняется запрос, страница: 1
Выполняется запрос, страница: 2
Выполняется запрос, страница: 3
Выполняется запрос, страница: 4
Выполнение завершено.


In [6]:
for vacancy in vacancies:
    print(vacancy['title'])
    print(vacancy['salary'])
    print(vacancy['salary_raw'])
    print(vacancy['ref'])
    print(vacancy['platform'])
    print('_'*25)

Бухгалтер
{'min': '35000', 'max': '45000', 'currency': 'руб.', 'comment': None}
35 000 — 45 000 руб.
https://nn.rabota.ru/vacancy/vacancy/35485411/?search_id=1663520004703349itfwf78r
nn.rabota.ru
_________________________
Главный бухгалтер
{'min': '78850', 'max': None, 'currency': None, 'comment': 'От'}
от 78 850 руб.
https://nn.rabota.ru/vacancy/vacancy/46942440/?search_id=1663520004703349itfwf78r
nn.rabota.ru
_________________________
Помощник главного бухгалтера
{'min': '30000', 'max': '40000', 'currency': 'руб.', 'comment': None}
30 000 — 40 000 руб.
https://nn.rabota.ru/vacancy/vacancy/46869275/?search_id=1663520004703349itfwf78r
nn.rabota.ru
_________________________
Главный бухгалтер
{'min': '40000', 'max': None, 'currency': None, 'comment': 'От'}
от 40 000 руб.
https://nn.rabota.ru/vacancy/vacancy/42954565/?search_id=1663520004703349itfwf78r
nn.rabota.ru
_________________________
Бухгалтер на участок
{'min': '40000', 'max': '40000', 'currency': None, 'comment': None}
40 000 руб

In [9]:
with open('vacancies', 'wb') as f:
    pickle.dump(vacancies, f)